<a href="https://colab.research.google.com/github/Ankitz007/Keras_basics/blob/master/NLP_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [0]:
import numpy as np

In [0]:
train_data, test_data = imdb['train'], imdb['test']

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [0]:
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [0]:
from keras import Sequential

Using TensorFlow backend.


In [0]:
type(training_labels[0])

numpy.int64

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# setting the parameters
vocab_size = 10000
max_length = 120
embedding_dim = 16
oov_tok = "XDX"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating='post', padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [0]:
len(word_index)
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you XDX into a big arm chair and XDX for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no XDX to cross no dangerous waters just a warm and witty XDX through new york life at its best a family film in every sense and one that deserves the praise it received ' ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [0]:
# look out for the embedding and globalaveragepooling1d layer
model = keras.Sequential([
                          keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                          keras.layers.GlobalAveragePooling1D(),
                          keras.layers.Dense(10, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 160,181
Trainable params: 160,181
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer=tf.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded, training_labels_final,epochs=10,validation_data=(testing_padded, testing_labels_final), verbose=1)

Train on 25001 samples, validate on 50000 samples
Epoch 1/10
25001/25001 [==============================] - 6s 227us/step - loss: 0.5521 - accuracy: 0.7265 - val_loss: 0.3853 - val_accuracy: 0.8409
Epoch 2/10
25001/25001 [==============================] - 4s 158us/step - loss: 0.3270 - accuracy: 0.8652 - val_loss: 0.3315 - val_accuracy: 0.8575
Epoch 3/10
25001/25001 [==============================] - 4s 157us/step - loss: 0.2654 - accuracy: 0.8932 - val_loss: 0.3335 - val_accuracy: 0.8569
Epoch 4/10
25001/25001 [==============================] - 4s 156us/step - loss: 0.2303 - accuracy: 0.9125 - val_loss: 0.3564 - val_accuracy: 0.8478
Epoch 5/10
25001/25001 [==============================] - 4s 158us/step - loss: 0.2041 - accuracy: 0.9242 - val_loss: 0.3772 - val_accuracy: 0.8444
Epoch 6/10
25001/25001 [==============================] - 4s 158us/step - loss: 0.1837 - accuracy: 0.9327 - val_loss: 0.4038 - val_accuracy: 0.8404
Epoch 7/10
25001/25001 [==============================] - 4s 1

In [0]:
model.evaluate(testing_padded,testing_labels_final)

50000/50000 [==============================] - 2s 35us/step


[0.5443006337738037, 0.8202000260353088]

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
sentence = "I fucking think this is amazing sculpture. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 1, 105, 12, 7, 491, 1216]]


# **For visualization purpose only**


In [0]:
import io
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')